In [1]:
# Adult Income — Feedforward DNN + fairness metrics
# Input :  data/adult_model.csv
# Output:  results/metrics/adult_dnn_overall.csv
#          results/metrics/adult_dnn_groups.csv
#          models/adult/dnn_model.keras

from pathlib import Path
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Paths
project_root = Path.cwd().resolve().parent  # run from notebooks/
data_dir = project_root / "data"
results_dir = project_root / "results" / "metrics"
models_dir = project_root / "models" / "adult"
results_dir.mkdir(parents=True, exist_ok=True)
models_dir.mkdir(parents=True, exist_ok=True)

# Load
df = pd.read_csv(data_dir / "adult_model.csv")

# Target and features
y = df["label"].astype(int).values
X = df.drop(columns=["label"]).copy()

# Sensitive for fairness reporting
sensitive_cols = [c for c in ["sex", "race"] if c in X.columns]
sens_all = X[sensitive_cols].copy() if sensitive_cols else pd.DataFrame(index=X.index)

# Split
X_train, X_test, y_train, y_test, sens_train, sens_test = train_test_split(
    X, y, sens_all, test_size=0.25, random_state=42, stratify=y
)

# Preprocess: numeric vs categorical
num_cols = [c for c in X_train.columns if np.issubdtype(X_train[c].dtype, np.number)]
cat_cols = [c for c in X_train.columns if c not in num_cols]

# Use sparse_output for newer sklearn
ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=True)
preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(with_mean=False), num_cols),
        ("cat", ohe, cat_cols),
    ],
    remainder="drop",
    sparse_threshold=1.0,
)

X_train_m = preprocess.fit_transform(X_train)
X_test_m  = preprocess.transform(X_test)

# Convert sparse → dense for Keras
X_train_m = X_train_m.toarray()
X_test_m  = X_test_m.toarray()

# DNN
model = keras.Sequential([
    layers.Input(shape=(X_train_m.shape[1],)),
    layers.Dense(64, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(32, activation="relu"),
    layers.Dense(1, activation="sigmoid"),
])
model.compile(optimizer=keras.optimizers.Adam(1e-3),
              loss="binary_crossentropy",
              metrics=["accuracy"])

callbacks = [keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)]

history = model.fit(
    X_train_m, y_train,
    validation_split=0.2,
    epochs=50,
    batch_size=512,
    callbacks=callbacks,
    verbose=1
)

# Predictions
y_prob = model.predict(X_test_m).ravel()
y_pred = (y_prob >= 0.5).astype(int)

from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
overall = {
    "model": "dnn",
    "accuracy": accuracy_score(y_test, y_pred),
    "f1": f1_score(y_test, y_pred),
    "roc_auc": roc_auc_score(y_test, y_prob),
}

# Fairness per group
rows = []
if not sens_test.empty:
    def tpr(y_true, y_hat):
        pos = (y_true == 1).sum()
        return ((y_true == 1) & (y_hat == 1)).sum() / pos if pos > 0 else np.nan

    for attr in sens_test.columns:
        groups = sens_test[attr].astype(str).unique()
        per_group = []
        for g in sorted(groups):
            mask = sens_test[attr].astype(str) == g
            yp_g = y_pred[mask]; yt_g = y_test[mask]
            p_pos = (yp_g == 1).mean() if len(yp_g) else np.nan
            acc = accuracy_score(yt_g, yp_g) if len(yt_g) else np.nan
            tpr_g = tpr(yt_g, yp_g) if len(yt_g) else np.nan
            per_group.append({"p_pos": p_pos, "tpr": tpr_g})

            rows.append({
                "model": "dnn", "attribute": attr, "group": g,
                "n": int(mask.sum()), "p_pos": p_pos, "accuracy": acc, "tpr": tpr_g
            })

        dp_gap = np.nanmax([r["p_pos"] for r in per_group]) - np.nanmin([r["p_pos"] for r in per_group])
        eopp_gap = np.nanmax([r["tpr"] for r in per_group]) - np.nanmin([r["tpr"] for r in per_group])
        overall[f"{attr}_dp_gap"] = dp_gap
        overall[f"{attr}_eopp_gap"] = eopp_gap

# Save metrics + model
overall_df = pd.DataFrame([overall])
groups_df = pd.DataFrame(rows) if rows else pd.DataFrame(columns=[
    "model","attribute","group","n","p_pos","accuracy","tpr"
])

overall_path = results_dir / "adult_dnn_overall.csv"
groups_path  = results_dir / "adult_dnn_groups.csv"
overall_df.to_csv(overall_path, index=False)
groups_df.to_csv(groups_path,  index=False)
print("Overall metrics:\n", overall_df.round(4))
print("\nSaved:", overall_path)
if not groups_df.empty:
    print("\nSample group metrics:\n", groups_df.head(12))
    print("Saved:", groups_path)

model_path = models_dir / "dnn_model.keras"
model.save(model_path)
print("\nSaved model:", model_path)


Epoch 1/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 13ms/step - accuracy: 0.7868 - loss: 0.4637 - val_accuracy: 0.8306 - val_loss: 0.3693
Epoch 2/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8357 - loss: 0.3561 - val_accuracy: 0.8404 - val_loss: 0.3391
Epoch 3/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8449 - loss: 0.3346 - val_accuracy: 0.8426 - val_loss: 0.3325
Epoch 4/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8491 - loss: 0.3287 - val_accuracy: 0.8417 - val_loss: 0.3321
Epoch 5/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8506 - loss: 0.3242 - val_accuracy: 0.8433 - val_loss: 0.3283
Epoch 6/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8525 - loss: 0.3209 - val_accuracy: 0.8449 - val_loss: 0.3268
Epoch 7/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8540 - loss: 0.3177 - val_accuracy: 0.8464 - val_loss: 0.3267
Epoch 8/50
53/53 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8522 - loss: 0.3167 - val_accuracy: 0.8464 - val_loss